<a href="https://colab.research.google.com/github/pedroandre1712/portfolio/blob/main/energy_bill_pdf_extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Esse código foi um projeto para a ONG Revolusolar.
O objetivo desse código é coletar algumas informações de forma automatizada das contas de luz dos cooperados e criar um arquivo excel contendo todas esses dados.

As informações que serão coletadas são

▶ *kWh consumidos*

▶ *preco unitario da energia eletrica*

▶ *saldo total de credito*

▶ *energia injetada*

▶ *energia injetada no ciclo*

▶ *subtotal outros*

▶ *total a pagar*

▶ *leitura estimada* 


# Algoritmo para realizar a coleta de dados

## Conectando com Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Instalando bibliotecas ✅

In [ ]:
pip install PyPDF2

     |████████████████████████████████| 77 kB 2.6 MB/s 
  Created wheel for PyPDF2: filename=PyPDF2-1.26.0-py3-none-any.whl size=61101 sha256=b48e53f01b449f90a01776677ee179d501f5d982904aeb2b906d1921e60ea73e
  Stored in directory: /root/.cache/pip/wheels/80/1a/24/648467ade3a77ed20f35cfd2badd32134e96dd25ca811e64b3
Successfully built PyPDF2


## Importando bibliotecas ✅

In [ ]:
from PyPDF2 import PdfFileReader
import pandas as pd
from os import listdir
from os.path import isfile, join

## Criacao das funcoes para realizar a coleta

### kWh consumidos ✅

In [ ]:
def coleta_kwhconsumidos(texto):
  read = texto.find('kWh', 24)
  consumo = texto[read+3:texto.find(',')-1]
  return consumo

### Preco unitario da energia eletrica ✅

In [ ]:
def coleta_unitario(texto):
  virgula = texto.find(',')
  tarifa = texto[virgula-1:virgula+6]
  return tarifa

### Saldo total de credito ✅


In [ ]:
def coleta_saldocredito(texto):
  read = texto[texto.find('Micro'):texto.find('Micro')+90]
  readtres = read.find('crédito')
  readquatro = read.find('Conven')
  saldo = read[readtres+len('crédito'):readquatro]
  if '.' in saldo:
    saldo.replace('.','')
  saldo = int(saldo)
  return saldo

### Energia injetada ✅

In [ ]:
def coleta_energiainjetada(texto):
  ancora = 'Energia Injetada kWh - TUSD'
  read = texto[texto.find(ancora):texto.find(ancora)+40]
  energia_inj = read[read.find('TUSD')+12:read.find(',')-1]
  return energia_inj

### Energia injetada no ciclo ✅

In [ ]:
def coleta_energiainjetadanociclo(texto): 
  read = texto[texto.find('Micro'):texto.find('Micro')+90]
  readum = read.find('ciclo')
  readdois = read.find('Saldo')
  readtres = read.find('crédito')
  readquatro = read.find('Conven')
  energia_injetada = read[readum+len('ciclo'):readdois]
  energia_injetada = int(energia_injetada)
  return energia_injetada

### Subtotal outros ✅

In [ ]:
def coleta_subtotaloutros(texto):
  fragmento = texto[texto.find('Outros')+len('Outros'):texto.find('ICMS')]
  indice = fragmento.find(',')
  subtotal_outros = fragmento[indice+3:fragmento.find(',',indice+1)+3]
  return subtotal_outros

### Total a pagar ✅

In [ ]:
def coleta_totalapagar(texto, year, month):
  ancora = month+'/'+year
  i_ancora = texto.find(ancora)
  i_total_a_pagar = texto[i_ancora:].find('$')+i_ancora
  i_virgula = texto[i_total_a_pagar:].find(',')+i_total_a_pagar
  total_a_pagar = texto[i_total_a_pagar+2:i_virgula+3]
  return total_a_pagar

### Leitura Estimada ✅

In [ ]:
def coleta_leituraestimada(texto):
  if 'LEITURA ESTIMADA' in texto:
    return 1
  else:
    return 0

## Coleta

In [ ]:
df = pd.DataFrame({'codigo':[],
                   'energia_injetada_no_ciclo':[],
                    'energia_injetada':[],
                    'saldo_credito':[],
                    'leitura_estimada':[],
                    'kWh_consumidos':[],
                    'tarifa':[],
                    'subtotal':[],
                    'total_a_pagar':[]
                     })


mes = ['JAN', 'FEV', 'MAR', 'ABR','MAI', 'JUN', 
       'JUL', 'AGO', 'SET', 'OUT', 'NOV', 'DEZ']

dicionario = {}

for i, mes in enumerate(mes):
  dicionario[mes] = i+1

ano = input('Digite o ano que deseja consultar: ')
mes = input('Digite o mes que deseja consultar: ').upper()
pasta_ano_mes = ano+'.'+str(dicionario[mes])
path = '/content/drive/MyDrive/Contas Light cada cooperado/'+pasta_ano_mes+'/'
print(path)
files = [f for f in listdir(path) if isfile(join(path, f))]

for file in files:
    with open(path+file,'rb') as caminho:

      

      pdf = PdfFileReader(caminho)
      page = pdf.getPage(0)
      text = page.extractText()

      codigo = file[:7]
      kwh_consumidos = coleta_kwhconsumidos(text)
      unitario = coleta_unitario(text)
      saldo_credito = coleta_saldocredito(text)
      energia_injetada = coleta_energiainjetada(text)
      energia_injetada_no_ciclo = coleta_energiainjetadanociclo(text)
      subtotal_outros = coleta_subtotaloutros(text)
      total_a_pagar = coleta_totalapagar(text, ano, mes)
      leitura_estimada = coleta_leituraestimada(text)

      dicionario = {'codigo':codigo,
                    'energia_injetada_no_ciclo':energia_injetada_no_ciclo,
                        'energia_injetada':energia_injetada,
                        'saldo_credito':saldo_credito,
                        'leitura_estimada':leitura_estimada,
                        'kWh_consumidos': kwh_consumidos,
                        'tarifa':unitario,
                        'subtotal':subtotal_outros,
                        'total_a_pagar':total_a_pagar
                        }
      df = df.append(dicionario, ignore_index = True)
df.to_excel(path+'output_teste.xlsx')

NameError: ignored

In [ ]:
b = pd.read_excel(path + 'output_teste.xlsx')
b[['codigo', 'subtotal']]

To Do

Colocar o path da revolusolar ✅

Automatizar com o ano e a data ✅

mor_012

